# Compute Results

This notebook reads the predictions of a model and computes the AUCs for each desease

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import roc_auc_score, roc_curve

n_splits = 10  # Number of randomized splits to compute standard deviations
split = ShuffleSplit(n_splits=n_splits, test_size=0.5, random_state=0)

pathologies = ["Atelectasis", "Consolidation", "Infiltration",
               "Pneumothorax", "Edema", "Emphysema", "Fibrosis", "Effusion", "Pneumonia",
               "Pleural_Thickening", "Cardiomegaly", "Nodule", "Mass", "Hernia"]

original_paper_results = ["0.8094", "0.7901", "0.7345",
                          "0.8887", "0.8878", "0.9371", "0.8047", "0.8638", "0.7680",
                          "0.8062", "0.9248", "0.7802", "0.8676", "0.9164"]

In [17]:
all_labels_path = "/home/user1/Desktop/Plots/server_model_arnoweng_pad/all_labels_8.npy"
all_outputs_path = "/home/user1/Desktop/Plots/server_model_arnoweng_pad/all_outputs_8.npy"

In [18]:
all_labels = np.load(all_labels_path)
all_outputs = np.load(all_outputs_path)

In [19]:
print(all_outputs.shape)
all_labels = all_labels[~np.isnan(all_outputs).any(axis=1)]
all_outputs = all_outputs[~np.isnan(all_outputs).any(axis=1)]
print(all_outputs.shape)

(30000, 14)
(30000, 14)


In [20]:
print("# Results\n\n| desease | them  |  us  |\n|---|---|---|")

for i in range(14):
    if (all_labels[:, i] == 0).all():
        print("|", pathologies[i], "|", original_paper_results[i], "|", "ERR |")
    else:
        # Compute AUC and STD with randomized splits
        split_auc = [roc_auc_score(all_labels[split_index, i][np.isfinite(all_labels[split_index, i])],
                                   all_outputs[split_index, i][np.isfinite(all_labels[split_index, i])])
                     for split_index, _ in split.split(all_outputs) if
                     not (all_labels[split_index, i] == 0).all()]

        auc = np.mean(split_auc)
        std = np.std(split_auc)

        print("|", pathologies[i], "|", original_paper_results[i], "|",
              str(auc)[:6], "+-", str(std)[:6], "|")



# Results

| desease | them  |  us  |
|---|---|---|
| Atelectasis | 0.8094 | 0.8355 +- 0.0050 |
| Consolidation | 0.7901 | 0.8868 +- 0.0086 |
| Infiltration | 0.7345 | 0.6597 +- 0.0089 |
| Pneumothorax | 0.8887 | 0.8310 +- 0.0143 |
| Edema | 0.8878 | 0.9450 +- 0.0063 |
| Emphysema | 0.9371 | 0.7951 +- 0.0101 |
| Fibrosis | 0.8047 | 0.8186 +- 0.0165 |
| Effusion | 0.8638 | 0.9491 +- 0.0039 |
| Pneumonia | 0.7680 | 0.7866 +- 0.0054 |
| Pleural_Thickening | 0.8062 | 0.7574 +- 0.0073 |
| Cardiomegaly | 0.9248 | 0.9202 +- 0.0026 |
| Nodule | 0.7802 | 0.7120 +- 0.0093 |
| Mass | 0.8676 | 0.8293 +- 0.0137 |
| Hernia | 0.9164 | 0.8328 +- 0.0040 |
